In [5]:
#One of the biggest problems in the Des Moines Parks & Rec Analysis Project was determining sex,
#an attribute not disclosed from the data source.
#This notebook's project is to use NLTK and Naive Bayes to (try to) determine sex from a name reliably.
#Sources:
#http://www.nltk.org/_modules/nltk/classify/util.html
#http://stackoverflow.com/questions/16323078/gender-identification-in-natural-language-processing
#https://gist.github.com/vinovator/6e5bf1e1bc61687a1e809780c30d6bf6
#https://gist.github.com/dannguyen/fbdfd51e62941edf9d81

import nltk
import pandas
import numpy as np
from nltk import NaiveBayesClassifier
from nltk.corpus import names
import random
from sklearn.model_selection import train_test_split

#only run the NLTK download the first run to download the names corpus.
#nltk.download()


#returns the attribute to use
def get_attribs(n):
    return{"last_letter": n[-1]}
    
name_list = []
for name in names.words("female.txt"):
    name_list += [(name,"female")]
for name in names.words("male.txt"):
    name_list += [(name,"male")]

random.shuffle(name_list)

#tried to get a pandas dataframe to work, didn't quite get it. Future fix.
d = []
for name in names.words("female.txt"):
    d.append({'Name': name, 'Gender': "female"})
df = pandas.DataFrame(d)
#print(df["Name"])

#investigate this later
feature_sets = [(get_attribs(n), gender)
                    for (n, gender) in name_list]

#split into training and test set
(train_data, test_data)= train_test_split(feature_sets, test_size = 0.2)

nb = nltk.NaiveBayesClassifier.train(train_data)

print(nltk.classify.accuracy(nb, test_data))

print(nb.show_most_informative_features(5))




0.7533039647577092
Most Informative Features
             last_letter = 'k'              male : female =     39.0 : 1.0
             last_letter = 'a'            female : male   =     31.8 : 1.0
             last_letter = 'f'              male : female =     15.7 : 1.0
             last_letter = 'p'              male : female =     11.7 : 1.0
             last_letter = 'm'              male : female =      9.7 : 1.0
None


In [7]:
#Notable bug - uppercase will almost always predict female. ToDo.
csv_in = pandas.read_csv("cleaned_data.csv")
genders = []
for i in csv_in.index:
    genders += [nb.classify(get_attribs(csv_in.ix[i]["Name"].split()[0].lower()))]
csv_in["Gender"] = genders
print(csv_in[0:10])

   Unnamed: 0                      Name                           Title  \
0           0             BENJAMIN PAGE             Park & Rec Director   
1           1             KEVIN C MORAN             Senior Park Planner   
2           2             RONALD S WARD                   Parks Manager   
3           3       MATTHEW A SALVATORE          Administrative Analyst   
4           4            BRIAN H BECKER       Park & Rec Superintendent   
5           5            JOHN A HAGENER       Park & Rec Superintendent   
6           6         RICHARD E LEFLEUR  Buildings & Grounds Superintnd   
7           7  MARLENE ANDERSON-BEATTIE           Park & Rec Supervisor   
8           8           KIRK J ROBINSON   Bldg & Parks Maint Supervisor   
9           9         JENNIFER FLETCHER            Marketing Supervisor   

           Wage     Rate  Years  Gender  
0  $119,766.40   $57.60     5.8    male  
1   $87,469.20   $42.10    12.5    male  
2   $87,276.80   $42.00     1.1    male  
3   $8

In [9]:
csv_in.to_csv("employee_data_plus_gender.csv")

